LSTM - (LONG-SHORT TERM MEMORY) 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

EDIT FROM HERE

In [2]:
# Load your dataset
data = pd.read_csv(r'C:\\Users\manoj\Downloads\Major project data\Major pro source codes\DATASETS\filtered_indices_output.csv')  # Use raw string literal

# Ensure the dataset contains the required features
# Assuming your dataset has columns: 'stoploss', 'entry', 'closing'
data = data

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define the number of time steps
time_step = 7  # For one week

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
# Create sequences for training
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])  # Use all features
        y.append(data[i + time_step, :])      # Predict all features
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])  # Reshape for LSTM [samples, time steps, features]

In [ ]:
# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=3))  # Output layer for 3 features

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)


In [ ]:
# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Inverse scaling

In [ ]:
# Create a DataFrame for actual and predicted values
predicted_df = pd.DataFrame(data=predictions, columns=['stoploss', 'entry', 'closing'])
actual_df = pd.DataFrame(data=scaler.inverse_transform(y_test), columns=['stoploss', 'entry', 'closing'])

In [ ]:
# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(actual_df['closing'], label='Actual Closing Prices', color='blue')
plt.plot(predicted_df['closing'], label='Predicted Closing Prices', color='red')
plt.title('Actual vs Predicted Closing Prices')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()